In [1]:
import os
import re

import numpy as np
import pandas as pd
from video_and_webcam_inference import SyringeVolumeEstimator

In [2]:
final_dataframe= pd.DataFrame(columns=['video_path', 'actual_volume (ml)', 'min', 'max', 'mean', 'std', 'median', 'SEM', 'CV', 'diameter (cm)', 'zoom'])

In [3]:
# find all the video files in the "videos" directory and subfolders
video_paths = []

for root, dirs, files in os.walk("videos"):
    for file in files:
        if file.endswith(".mov"):
            video_paths.append(os.path.join(root, file))

print(f"Found {len(video_paths)} video files.")

Found 30 video files.


In [4]:
# Define the regular expression pattern.
pattern_diameter: re.Pattern = re.compile(r'/(\d+\.\d+)-\d+ml\.mov$')

# a pattern for finding the volume in the filename
pattern_volume: re.Pattern = re.compile(r'/\d+\.\d+-(\d+)ml\.mov$')

# a pattern for finding the zoom in the filename
pattern_zoom: re.Pattern = re.compile(r'^videos/(\d+x)/')

In [5]:
# Initialize an empty list to collect new rows
new_rows = []

for video_path in video_paths:
    print(f"Processing {video_path}...")

    # Create an instance of the SyringeVolumeEstimator
    estimator = SyringeVolumeEstimator()
    estimator.run(input_source='video', video_path=video_path)

    data = pd.read_csv('syringe_data.csv').iloc[0:150]

    # Extract diameter
    match_diameter = pattern_diameter.search(video_path)
    diameter: str = match_diameter.group(1) if match_diameter else "Unknown"

    # Extract volume
    match_volume = pattern_volume.search(video_path)
    volume: str = match_volume.group(1) if match_volume else "Unknown"

    # Extract zoom
    match_zoom = pattern_zoom.search(video_path)
    zoom: str = match_zoom.group(1) if match_zoom else "Unknown"

    column = f"volume_D{diameter}"

    # Calculate statistics
    min_val = data[column].min()
    max_val = data[column].max()
    mean_val = data[column].mean()
    std_val = data[column].std()
    median_val = data[column].median()
    sem_val = std_val / np.sqrt(len(data[column]))
    cv_val = (std_val / mean_val) * 100

    print(f"Min: {min_val:.3f}")
    print(f"Max: {max_val:.3f}")
    print(f"Mean: {mean_val:.3f}")
    print(f"Std: {std_val:.3f}")
    print(f"Median: {median_val:.3f}")
    print(f"SEM: {sem_val:.3f}")
    print(f"CV: {cv_val:.3f}")

    # Append the new row as a dictionary to the list
    new_rows.append({
        'video_path': video_path,
        'actual_volume (ml)': volume,
        'diameter (cm)': diameter,
        'min': min_val,
        'max': max_val,
        'mean': mean_val,
        'std': std_val,
        'median': median_val,
        'SEM': sem_val,
        'CV': cv_val,
        'zoom': zoom
    })

    print(f"Finished processing {video_path}.\n\n")

# After the loop, create a DataFrame from the list of new rows
new_rows_df = pd.DataFrame(new_rows)

# Concatenate the original DataFrame with the new rows DataFrame
final_dataframe = pd.concat([final_dataframe, new_rows_df], ignore_index=True)

# Display the last few rows of the updated DataFrame
display(final_dataframe.tail(30))

Processing videos/3x/1.25-2ml.mov...
🎥 Video resolution: 2160 x 3840, FPS: 60.22304832713755
Min: 2.060
Max: 2.274
Mean: 2.197
Std: 0.033
Median: 2.197
SEM: 0.003
CV: 1.511
Finished processing videos/3x/1.25-2ml.mov.


Processing videos/3x/1.25-0ml.mov...
🎥 Video resolution: 2160 x 3840, FPS: 60.09058882737796
Min: 0.073
Max: 0.165
Mean: 0.129
Std: 0.019
Median: 0.133
SEM: 0.002
CV: 14.427
Finished processing videos/3x/1.25-0ml.mov.


Processing videos/3x/1.25-4ml.mov...
🎥 Video resolution: 2160 x 3840, FPS: 60.18641810918775
Min: 3.785
Max: 3.976
Mean: 3.853
Std: 0.053
Median: 3.831
SEM: 0.004
CV: 1.382
Finished processing videos/3x/1.25-4ml.mov.


Processing videos/3x/2.0-5ml.mov...
🎥 Video resolution: 2160 x 3840, FPS: 60.151133501259444
Min: 4.681
Max: 5.345
Mean: 5.172
Std: 0.105
Median: 5.185
SEM: 0.009
CV: 2.028
Finished processing videos/3x/2.0-5ml.mov.


Processing videos/3x/2.0-0ml.mov...
🎥 Video resolution: 2160 x 3840, FPS: 60.1981833195706
Min: 0.135
Max: 3.269
Mean: 2.021

/var/folders/ht/hpy25npj1t96b4yb_vxvc_b00000gn/T/ipykernel_7240/2614856288.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, new_rows_df], ignore_index=True)


,video_path,actual_volume (ml),min,max,mean,std,median,SEM,CV,diameter (cm),zoom
0,videos/3x/1.25-2ml.mov,2,2.059908,2.273986,2.197226,0.033191,2.196692,0.002710,1.510606,1.25,3x
1,videos/3x/1.25-0ml.mov,0,0.072512,0.165290,0.128625,0.018557,0.133230,0.001515,14.426968,1.25,3x
2,videos/3x/1.25-4ml.mov,4,3.785489,3.975990,3.852884,0.053242,3.830676,0.004347,1.381880,1.25,3x
3,videos/3x/2.0-5ml.mov,5,4.681353,5.345304,5.172173,0.104916,5.184757,0.008566,2.028471,2.0,3x
4,videos/3x/2.0-0ml.mov,0,0.135026,3.268971,2.020592,0.853873,2.247073,0.069718,42.258527,2.0,3x
5,videos/3x/1.25-1ml.mov,1,0.839656,1.115004,1.067339,0.050770,1.080303,0.004145,4.756655,1.25,3x
6,videos/3x/2.0-20ml.mov,20,21.551167,22.436163,21.824073,0.139444,21.817061,0.011386,0.638945,2.0,3x
7,videos/3x/1.25-3ml.mov,3,3.053496,3.216981,3.091653,0.029524,3.080863,0.002411,0.954944,1.25,3x
8,videos/3x/1.25-5ml.mov,5,4.649082,4.953873,4.724696,0.070405,4.693694,0.005749,1.490145,1.25,3x
9,videos/3x/1.0-0ml.mov,0,0.025040,0.099645,0.059575,0.012887,0.060692,0.001052,21.632148,1.0,3x
